In [1]:
from importlib import reload
# Tips to me:
# Use Python 3.11.6
# That's in the location /opt/homebrew/bin/python3

# Libraries
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import dataset
from IPython.display import Audio
from Nets import complicatedNet
from torch.nn import functional as F
import models
import trainer

In [2]:
reload(dataset)
testset = dataset.AudioDatasetAt("datasets/clips-test")
trainset = dataset.AudioDatasetAt("datasets/clips-train")
validationset = dataset.AudioDatasetAt("datasets/clips-validation")


In [3]:
reload(dataset)
# Check Loaders
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, 8, shuffle=True, collate_fn=dataset.my_collater)
validationloader = DataLoader(validationset, 8, shuffle=True, collate_fn=dataset.my_collater)
testloader = DataLoader(testset, 8, shuffle=True, collate_fn=dataset.my_collater)

In [4]:
# Getting a random data
wf = next(testloader._get_iterator())
inp = wf

print(inp.shape)
Audio(inp[0], rate=32000)

torch.Size([18, 1, 96000])


In [14]:
# Instantiating a model
reload(complicatedNet)
reload(models)

encoder = complicatedNet.Encoder(4)
quantizer = torch.nn.Identity()
decoder = complicatedNet.Decoder(4)
my_model = models.allNet(
    encoder=encoder, quantizer=quantizer, decoder=decoder
)
device = 'cpu'
my_model.to(device)
dummydata = torch.rand([np.random.randint(21),1,256]).to(device)
my_model.eval()

my_model(dummydata)

reload(trainer)
lossfn = complicatedNet.LossFunction().to(device)
optimizer = trainer.Adam(my_model.parameters())

print(my_model)

allNet(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Down(
        (InConv): Conv1d(1, 16, kernel_size=(1,), stride=(1,))
        (Conv1): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(16, 16, kernel_size=(11,), stride=(1,))
          )
        )
        (Conv2): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(16, 16, kernel_size=(11,), stride=(1,), dilation=(2,))
          )
        )
        (Conv3): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(16, 16, kernel_size=(11,), stride=(1,), dilation=(4,))
          )
        )
        (Conv4): ConvolutionalStack(
          (ELU1): ELU(alpha=1.0)
          (Conv1): LazyCausalConv1d(
            (lazyLayer): Conv1d(16, 16, kernel_size=(11,), stride=(1,), dilation=(8,))
          )
        )
        (Pooli

In [ ]:
x = inp.to(device)
output = my_model(    x
    )[0].cpu().squeeze().unsqueeze(0).detach().numpy()

print(x.shape)
print(output.shape)
print("Before Training, Model is :")
Audio(output, rate=32000)


torch.Size([18, 1, 96000])
(1, 96000)
Before Training, Model is :


In [17]:
reload(trainer)
time, loss = trainer.train_one_epoch(my_model, lossfn, optimizer, trainloader, True)

Training one epoch
tensor(9337.0791, grad_fn=<MseLossBackward0>)
Trained on batch 1
tensor(23441.1973, grad_fn=<MseLossBackward0>)
Trained on batch 2
tensor(2727733.7500, grad_fn=<MseLossBackward0>)
Trained on batch 3


In [ ]:
idx = 0
size = 13
remaining = lambda : size - idx
idx, remaining()

In [ ]:
idx += 2
idx, remaining()

In [ ]:
spectrogramer1 = torchaudio.transforms.MelSpectrogram(32000)
spectrogramer2 = torchaudio.transforms.MelSpectrogram(32000)

In [ ]:
image1 = spectrogramer1(inp).cpu().detach()
image2 = spectrogramer1(out).cpu().detach()
image1 = F.interpolate(image1, scale_factor=.1, mode='bilinear').squeeze()
image2 = F.interpolate(image2, scale_factor=.1, mode='bilinear').squeeze()
# image1 = F.interpolate(image1, mode='linear', scale_factor=10).squeeze()
image1.shape, image2.shape

In [ ]:
plt.figure(figsize=(20,7))
plt.figure(211)
plt.imshow(image1)
plt.figure(212)
plt.imshow(image2)